In [2]:
import numpy as np
import json
import os
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import scipy.stats as stats
import torch
import torch.nn.functional as F
from sklearn.preprocessing import normalize


In [3]:
CUDA = torch.cuda.is_available()
print("CUDA: %s" % CUDA)

CUDA: False


In [4]:
porter = PorterStemmer()
stop_words = set(stopwords.words("english"))

In [5]:
model_fld = "model"

In [214]:

def skipgram_sim(embedding, w2i,target, r_ids, context, type=None):
    
    #target : word to be replaced
    #r_ids : replacement words
    #context: context words
    
    t = embedding[target]
    r_ids = np.array([embedding[a] for a in r_ids])
    
    
    c_len = len(context)
    c = np.array([embedding[w] for w in context])
#     print(c.shape, r_ids.shape, t.shape)
    c = normalize(c, axis=1, norm='l2')
    r_ids = normalize(r_ids, axis=1, norm='l2')
    t = t/np.linalg.norm(t)
#     print(c.shape, r_ids.shape, t.shape)
#     print(r_ids @ t)
    if type == 'cosine':
            scores = r_ids @ t
            scores = scores.tolist()

    elif type == 'add':
            scores = [
                (a @ t + np.sum(c @ a)) / (c_len + 1)
            for a
            in r_ids
    ]
    elif type == 'mult':
        scores = [
                (((t @ a + 1) / 2) * np.prod((c @ a + 1) / 2)) ** (1 / (c_len + 1))
                for a
                in r_ids
        ]
        
    return scores

In [6]:
#Lexical substituion task


def Lst_subs(w2i, win_size = 2):
    
    #model_type: embedding or whole model
    with open('data/lst/lst.gold.candidates', 'r') as f:
        lines = map(str.strip, f.readlines())

    w2i = defaultdict(lambda: UNK, w2i)
    candidates = {}
    for line in lines:
        target, rest = line.split('.',maxsplit=1)
        pos_tag, rest = rest.split('::',maxsplit=1)
        replacement = rest.split(';')
        candidates[target] = replacement


    with open('data/lst/lst_test.preprocessed', 'r') as f:
        lines = map(str.strip, f.readlines())

    skip = 0
#     with open((os.path.join(model_fld,name+'_lst.out')), 'w') as f_out:
    t_all = []
    r_id_all =[]
    c_all =[]
    sent_all=[]
    rep=[]
    target_all=[]
    
    for line in lines:
        target, sent_id, t_pos, sentence = line.split('\t')

        target_word = target.split('.')[0]
        sentence = sentence.split()
        t_pos = np.int(t_pos)

        if target_word in w2i.keys():
            target_id = w2i[target_word]
        else:
            skip+= 1
            continue

        #get contexts around target positon which are not stopwords and Punctuation
        a_context = [w for w in sentence[t_pos+1:] if w.isalpha() if w not in stop_words] 
        b_context = [w for w in sentence[:t_pos][::-1] if w.isalpha() if w not in stop_words]
        
        if win_size != 0:
            context = b_context[:win_size]+a_context[:win_size]
            context_ids = [w2i[w] for w in context if w in w2i.keys()]
        else:
            context = [b_context[::-1], a_context]
            context_left = [w2i[w] for w in context[0] if w in w2i.keys()]
            context_right = [w2i[w] for w in context[1] if w in w2i.keys()]
            
            if len(context_left)==0 or len(context_right)==0:
                continue
            context_ids = [context_left, context_right]

        
        if len(context_ids)==0:
            continue

        replacement = candidates[target_word]
        r_ids = [w2i[w] for w in replacement if w in w2i.keys()]
        t_all.append(target_id)
        r_id_all.append(r_ids)
        c_all.append(context_ids)
        sent_all.append(sent_id)
        rep.append(replacement)
        target_all.append(target)
    return t_all, r_id_all, c_all, sent_all, rep, target_all
    
#             scores = func(model_type, w2i, target_id, r_ids, context_ids, type="kl")
#             break
#         #             print(scores)
            
#             print('RANKED\t{} {}'.format(target, sent_id), file=f_out, end='')

#             # Sort alternative by their scores
#             words_and_scores = list(zip(replacement, scores))
            
#             words_and_scores.sort(key=lambda t: t[1], reverse=True)

#             # Write ranked replacement and their scores to file
#             for w, s in words_and_scores:
#                 print('\t{} {}'.format(w, s), file=f_out, end='')
#             print(file=f_out)
#     print("done!!")

## SkipGram

In [232]:
def getout_skip(embedding,target_all, rep_all, context_all, sent_all, rep, word, metrics="cosine"):
    
    with open((os.path.join(model_fld,"skip_"+metrics+'_lst.out')), 'w') as f_out:
            for target_id, r_ids, context_ids, sent_id, replacement, target in zip(target_all, rep_all, \
                                                                           context_all, sent_all, rep, word):
#                 print(target_id, r_ids, context_ids, sent_id, replacement, target)
                scores =  skipgram_sim(embedding, w2i_skip, target_id, r_ids, context_ids, type=metrics)
#                 break

                print('RANKED\t{} {}'.format(target, sent_id), file=f_out, end='')

                # Sort alternative by their scores
                words_and_scores = list(zip(replacement, scores))

                words_and_scores.sort(key=lambda t: t[1], reverse=True)

                # Write ranked replacement and their scores to file
                for w, s in words_and_scores:
                    print('\t{} {}'.format(w, s), file=f_out, end='')
                print(file=f_out)

            

In [237]:
#load SkipGram
fl = "skipgram.npz"
with open(os.path.join(model_fld, "skipgram_w2i.txt"), 'rb') as f:
        w2i_skip = json.load(f)
embedding = np.load(os.path.join(model_fld, fl))["arr_0"]
target_all, rep_all, context_all, sent_all, rep, word = Lst_subs(w2i_skip, win_size=2)

getout_skip(embedding, target_all, rep_all, context_all, sent_all, rep,word,"cosine")
getout_skip(embedding, target_all, rep_all, context_all, sent_all, rep,word,"add")
getout_skip(embedding, target_all, rep_all, context_all, sent_all, rep,word,"mult")

## Bayesian SkipGram

In [7]:
def kldiv(mu, sig, mu_rep, sig_rep):
    t1 = (mu_rep)*0.5 - mu*0.5
    t2_num = (mu-mu_rep)**2 + torch.exp(mu)
    t2_den = 2*(torch.exp(sig_rep)) 
    t2 = ((t2_num/t2_den)-0.5)
    kl_loss = t2+t1
    kl_loss = torch.sum(kl_loss, dim=1)
    return kl_loss.cpu().detach().numpy()[0]
    

def bsg(model, w2i, target, r_ids, context, win_size = 2,type=None):
    #target : word to be replaced
    #r_ids : replacement words
    #context: context words
    
    model.eval()
    PAD = w2i["<pad>"]
    target = [target]
    
    score =[]
    
    if len(context)< win_size*2:
        context+=[PAD]*(win_size*2-len(context))
    #--------------------------------------
    #run a fwd pass for target word
    if CUDA:
        target  = torch.cuda.LongTensor(target)
        context = torch.cuda.LongTensor(context).unsqueeze(0)
        r_ids   = torch.cuda.LongTensor(r_ids)
    else:
        target  = torch.LongTensor(target)
        context = torch.LongTensor(context).unsqueeze(0)
        r_ids   = torch.LongTensor(r_ids)
    
    
    _, mu, sig,_, _ = model.forward(target, context, win_size)
    
   
    #---------------------------------------
    context = context.repeat(r_ids.size()[0],1)
    
    #run fwd pass for each of context word keeping same context
    _,mu_r, sig_r, _, _ = model.forward(r_ids, context,win_size)
    #----------------------------------------
    
    #---convert to numpy------
    #mu, sig = mu.cpu().detach().numpy(), sig.cpu().detach().numpy()
    ##-----------------------------
    #mu_r = mu_r.cpu().detach().numpy()
   
       
    #---------------------------
    
    
    if type == "kl":
        for mu_rep, sig_rep in zip(mu_r, sig_r):
            mu_rep = mu_rep.unsqueeze(0)
            sig_rep = sig_rep.unsqueeze(0)
            
            score.append(kldiv(mu, sig, mu_rep, sig_rep))
        
        return score    

In [234]:
def getout_bsg(model, target_all, rep_all, context_all, sent_all, rep, word,metrics=None, win_size=2):
    
    with open((os.path.join(model_fld,"bsg_"+metrics+'_lst.out')), 'w') as f_out:
            for target_id, r_ids, context_ids, sent_id, replacement, target in zip(target_all, rep_all, \
                                                                           context_all, sent_all, rep, word):
                scores =  bsg(model, w2i_skip, target_id, r_ids, context_ids, win_size, metrics)
#                 break

                print('RANKED\t{} {}'.format(target, sent_id), file=f_out, end='')

                # Sort alternative by their scores
                words_and_scores = list(zip(replacement, scores))

                words_and_scores.sort(key=lambda t: t[1], reverse=True)

                # Write ranked replacement and their scores to file
                for w, s in words_and_scores:
                    print('\t{} {}'.format(w, s), file=f_out, end='')
                print(file=f_out)

In [241]:
from bayesian_skip import BayesianGram

fl = "bsg"
with open(os.path.join(model_fld, "bsg_w2i.txt"), 'rb') as f:
        w2i_bsg = json.load(f)

target_all, rep_all, context_all, sent_all, rep, word = Lst_subs(w2i_skip, win_size=5)
model = torch.load(os.path.join(model_fld, fl))

getout_bsg(model, target_all, rep_all, context_all, sent_all, rep, word,metrics="kl", win_size = 5)


## EmbedAlign

In [8]:
def get_batch(words,context,sentence_length, batch_size, pad):
    target = words[0]
    r_ids = words[1]
    
    batch = torch.zeros(batch_size,sentence_length)
    
    for i in range(0,len(r_ids)+1):
        if i == 0:
            sentence = context[0] + [target] + context[1]
        else:
            sentence = context[0] + [r_ids[i-1]] + context[1]
        
        if len(sentence) < sentence_length:
            diff = sentence_length - len(sentence)
            pad_m = [pad]*diff
            sentence_add = sentence + pad_m
        else:
            sentence_add = sentence

        batch[i,:] = torch.Tensor(sentence_add)
        
    return batch.long()
    

In [9]:
# target = w2i, r_id=w2i, sentence=:::
def embedalign(model, w2i_embdL1, target, r_ids, context):
    hidden_dim = 100 # Use the same values as training
    batch_size = 128  # Use the same values as training
    sentence_length = 64 # Could use the same as training
    dim_Z = 150
    
    
    ffnn1 = models[0]
    ffnn2 = models[1]
    ffnn3 = models[2]
    ffnn4 = models[3]
    lstm = models[4]

    ffnn1.eval()
    ffnn2.eval()
    ffnn3.eval()
    ffnn4.eval()
    lstm.eval()
    
    
    words = [target, r_ids]
    pad = w2i_embdL1["<pad>"]
#     print("PAD",pad)
    
    data = get_batch(words,context,sentence_length, batch_size, pad)
    lstm.word_embeddings.eval()
#     print(data[0])
#     lstm.word_embeddings(data[0])
  
    if data.shape[0] != batch_size:
        required_size = batch_size - data.shape[0]
        dummy_data = torch.zeros(required_size,sentences)
        print(dummy_data.shape)
        data = torch.cat(data, dummy_data)
            

    # Because training was done in batch, the LSTM structure/model seems to be hardcoded for batch
    print(data.shape)
    h_1 = lstm(data)
    h = (h_1[:, :, 0:hidden_dim] + h_1[:, :, hidden_dim:]) / 2

    mu = ffnn3(h, linear_activation=True)
    sigma = ffnn4(h)

    return mu, sigma
        

In [10]:
def getout_embed(models, target_all, rep_all, context_all, sent_all, rep, word,metrics=None, win_size=0):

    with open((os.path.join(model_fld,"embed_"+metrics+'_lst.out')), 'w') as f_out:
            for target_id, r_ids, context_ids, sent_id, replacement, target in zip(target_all, rep_all, \
                                                                           context_all, sent_all, rep, word):
                mu, sigma =  embedalign(models, w2i_embdL1, target_id, r_ids, context_ids)

                target_pos = len(context_ids[0])
                mu_t = mu[0,target_pos,:]
                sigma_t = sigma[0,target_pos,:]
                scores = []
                for i in range(0,mu.shape[0]):
                    if i == 0:
                        continue
                    else:
                        mu_i = mu[i,target_pos,:]
                        sigma_i = sigma[i,target_pos,:]
                        
                    scores.append(kldiv(mu_t.unsqueeze(0), torch.log(sigma_t.unsqueeze(0)), mu_i.unsqueeze(0), \
                                        torch.log(sigma_i.unsqueeze(0))))
                    
                print('RANKED\t{} {}'.format(target, sent_id), file=f_out, end='')

                # Sort alternative by their scores
                words_and_scores = list(zip(replacement, scores))

                words_and_scores.sort(key=lambda t: t[1], reverse=True)

                # Write ranked replacement and their scores to file
                for w, s in words_and_scores:
                    print('\t{} {}'.format(w, s), file=f_out, end='')
                print(file=f_out)


In [11]:
def load_models():
    ffnn1 = torch.load('ffnn1.pt')
    ffnn2 = torch.load('ffnn2.pt')
    ffnn3 = torch.load('ffnn3.pt')
    ffnn4 = torch.load('ffnn4.pt')

    lstm = torch.load('lstm.pt')

    return [ffnn1, ffnn2, ffnn3, ffnn4, lstm]

In [12]:
# model_fld = "model_test"
model_fld = "model"
file_name = "L1_w2i_f.json"
with open(os.path.join(model_fld,file_name), 'rb') as f:
    w2i_embdL1 = json.load(f)
    
target_all, rep_all, context_all, sent_all, rep, word = Lst_subs(w2i_embdL1, win_size=0)

models = load_models()

getout_embed(models, target_all, rep_all, context_all, sent_all, rep, word,metrics="kl", win_size = 0)


PAD 0
tensor([  79,  102,   76,  173,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
torch.Size([128, 64])
PAD 0
tensor([ 100,   76,  100,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])
torch.Size([128, 64])
PAD 0
tensor([  99,  213,   79,    4,    0,    0,    0,    0, 

KeyboardInterrupt: 

In [18]:
print(len(w2i_embdL1))
for i in range(0,len(context_all)):
    context_l, context_r = context_all[i], context_all[i]
    print(context_l)
    print(context_r)


3455
[[135, 347, 1672], [1282, 2081]]
[[135, 347, 1672], [1282, 2081]]
[[531, 951, 1054], [2869, 203]]
[[531, 951, 1054], [2869, 203]]
[[1328, 1220], [541, 816, 689, 1176, 485, 138, 1461, 2220, 1176]]
[[1328, 1220], [541, 816, 689, 1176, 485, 138, 1461, 2220, 1176]]
[[910], [19, 20, 94, 241, 261, 910, 2428, 856, 1639]]
[[910], [19, 20, 94, 241, 261, 910, 2428, 856, 1639]]
[[519, 825, 870], [2403, 1271, 1857, 1727, 2120, 136]]
[[519, 825, 870], [2403, 1271, 1857, 1727, 2120, 136]]
[[2759, 1370, 3248], [2092]]
[[2759, 1370, 3248], [2092]]
[[1161], [470, 3138]]
[[1161], [470, 3138]]
[[304], [2429, 910, 964, 1146]]
[[304], [2429, 910, 964, 1146]]
[[238], [920, 1058, 1014, 3449, 847, 1833, 112, 723, 141, 1523, 1058]]
[[238], [920, 1058, 1014, 3449, 847, 1833, 112, 723, 141, 1523, 1058]]
[[55, 847, 100, 1800], [577, 1461, 1226, 634]]
[[55, 847, 100, 1800], [577, 1461, 1226, 634]]
[[2837], [41, 19, 1929, 2382, 1929, 19, 1220, 972]]
[[2837], [41, 19, 1929, 2382, 1929, 19, 1220, 972]]
[[2358, 2

In [10]:
hidden_dim = 100 # Use the same values as training
batch_size = 32  # Use the same values as training
sentence_length = 64 # Could use the same as training
dim_Z = 150


pad = w2i_embdL1["<pad>"]


embedalign(models, w2i_embdL1, target_all[0], rep_all[0], context_all[0])

(tensor([[[-6.5598e-02,  3.2560e-02, -3.6306e-02,  ...,  2.4451e-02,
            8.4843e-02, -3.9054e-02],
          [-2.3850e-02,  9.1610e-02, -3.4445e-02,  ...,  2.4387e-02,
            1.4966e-01,  1.2788e-02],
          [ 8.3172e-03,  2.6678e-02, -3.3374e-02,  ...,  3.6800e-02,
            1.1598e-01,  3.8721e-02],
          ...,
          [-1.2384e-02,  2.5157e-02, -2.6264e-02,  ...,  2.2478e-02,
            3.2349e-02, -9.6847e-03],
          [-2.2433e-02,  2.9369e-02, -2.4605e-02,  ...,  3.0274e-02,
            3.6437e-02, -1.5883e-03],
          [-3.7119e-02,  3.9321e-02, -1.4808e-02,  ...,  4.3631e-02,
            4.4480e-02,  1.4211e-02]],
 
         [[ 2.5193e-03,  1.8049e-02, -1.8573e-02,  ...,  8.1522e-03,
            2.7883e-02, -1.5968e-02],
          [ 2.4959e-03,  1.9185e-02, -1.9350e-02,  ...,  8.9958e-03,
            2.8335e-02, -1.6858e-02],
          [ 2.4050e-03,  1.9981e-02, -1.9860e-02,  ...,  9.5664e-03,
            2.8749e-02, -1.7492e-02],
          ...,
    

In [26]:
lstm = models[4]
lstm.